In [1]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [2]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [3]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools > /dev/null

In [4]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


In [5]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html  --quiet

     |████████████████████████████████| 735.4 MB 16 kB/s 
     |████████████████████████████████| 12.8 MB 33.6 MB/s 
     |████████████████████████████████| 7.6 MB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.1+cu101 which is incompatible.


In [6]:
!pip install transformers==3.5.1 --quiet

     |████████████████████████████████| 1.3 MB 2.1 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 2.9 MB 30.9 MB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 


In [7]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 3.60 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [8]:
!mkdir models/

# finetune

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 4 \
    --block_size 1024 \
    --overwrite_output_dir

In [ ]:
!zip weights.zip -r models

In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
model_name_or_path = 'models/essays'
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

In [23]:
import time

sent1 = "<speaker1>Я пошел гулять.<speaker2>"
sent2 = "<speaker1>Я чувствую обиду и злость за свой поступок.<speaker2>"
sent3 = "<speaker1>Снег автомату рознь, а собака скользкая.<speaker2>"
sent4 = "<speaker1>Скажи мне что-либо приятное.<speaker2>"

dataset = [sent1, sent2, sent3, sent4]

for item in dataset:
  text = item
  time_start = time.time()
  input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      top_k=1,
      top_p=0.95,
      temperature=1.2,
      num_return_sequences=1,
      max_length=32,
      no_repeat_ngram_size=3,
      repetition_penalty=2.5)
  generated_text = list(map(tokenizer.decode, out))[0]
  print("calc time {}".format(time.time() - time_start))
  print(generated_text)
  print("-----------------------------------")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


calc time 0.5626204013824463
<speaker1> Я пошел гулять. <speaker2> А я люблю готовить, особенно торты) <eos> <bos> "Привет! Как дела? Чем занимаешься?)"Добрый день!)
-----------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


calc time 0.4650156497955322
<speaker1> Я чувствую обиду и злость за свой поступок. <speaker2> А я люблю животных, поэтому у меня есть собака) а ты? <eos> <bos> "Привет! Как дела
-----------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


calc time 0.4066886901855469
<speaker1> Снег автомату рознь, а собака скользкая. <speaker2> А я люблю гулять по парку и смотреть на звезды) <eos> <bos> "Привет! Как дела
-----------------------------------
calc time 0.48810410499572754
<speaker1> Скажи мне что-либо приятное. <speaker2> Я люблю петь, а ты? <eos> <bos> "
Привет! Как дела?) Чем занимаешься?) Расскажи
-----------------------------------
